UNIFICAR VENTAS DEL MOSTRADOR Y FACTURAS


In [1]:
import sys
from pathlib import Path
import csv 
import pandas as pd
import re

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))

from main import unificar_ventas_excel_a_csv, parse_line
from utilidades.constantes import DATA_PATH, DATA_PROCESADA_PATH, PATRON_MOSTRADOR_PATH, MOSTRADOR_SALIDA_PATH, PATRON_FACTURACION_PATH, FACTURACION_SALIDA_PATH,ARCHIVO_UNIFICADO_PATH

UNIFICAR MOSTRADOR

In [2]:
unificar_ventas_excel_a_csv(
    DATA_PATH,
    PATRON_MOSTRADOR_PATH,
    MOSTRADOR_SALIDA_PATH
)

Leyendo: MOSTRADOR_1T_2023.xlsx
Leyendo: MOSTRADOR_1T_2024.xlsx
Leyendo: MOSTRADOR_2T_2023.xlsx
Leyendo: MOSTRADOR_2T_2024.xlsx
Leyendo: MOSTRADOR_3T_2023.xlsx
Leyendo: MOSTRADOR_3T_2024.xlsx
Leyendo: MOSTRADOR_4T_2023.xlsx
Leyendo: MOSTRADOR_4T_2024.xlsx
Leyendo: MOSTRADOR_ABRIL_2025.xlsx
Leyendo: MOSTRADOR_AGOSTO_2025.xlsx
Leyendo: MOSTRADOR_ENERO_2025.xlsx
Leyendo: MOSTRADOR_FEBRERO_2025.xlsx
Leyendo: MOSTRADOR_JULIO_2025.xlsx
Leyendo: MOSTRADOR_JUNIO_2025.xlsx
Leyendo: MOSTRADOR_MARZO_2025.xlsx
Leyendo: MOSTRADOR_MAYO_2025.xlsx
Archivo CSV final guardado en: ..\data_procesada\ventas_mostrador.csv


UNIFICAR FACTURAS 


In [3]:
data = []
for ventas_file in DATA_PATH.glob(PATRON_FACTURACION_PATH):
    with ventas_file.open(encoding="latin-1") as f:
        for i, line in enumerate(f, 1):
            parsed = parse_line(line, file_name=ventas_file.name, line_num=i)
            if parsed is not None:
                data.append(parsed)

df = pd.DataFrame(data)

if len(df) == 0:
    print("No se parseó ningún registro. Verificá el formato de los archivos.")
else:
    df["fecha_emision"] = pd.to_datetime(df["fecha_emision"], format="%Y%m%d", errors="coerce")
    df["fecha_emision"] = df["fecha_emision"].dt.strftime('%Y-%m-%d')
    df.to_csv(FACTURACION_SALIDA_PATH, index=False, encoding="utf-8-sig")
    print(f"¡Listo! {len(df)} comprobantes unificados en {FACTURACION_SALIDA_PATH}.")
    print("Columnas exportadas:", df.columns.tolist())


¡Listo! 2456 comprobantes unificados en ..\data_procesada\ventas_facturacion.csv.
Columnas exportadas: ['fecha_emision', 'tipo_comprobante', 'pto_venta', 'nro_comprobante', 'cuit_receptor', 'denominacion', 'importe_total', 'archivo_origen']


UNIFICAR MOSTRADOR CON FACTURAS (LIMPIAR ARCHIVOS PRIMERO)

In [4]:
import pandas as pd

# Dataset de resto de las ventas
df_resto = pd.read_csv(FACTURACION_SALIDA_PATH)

# Dataset del mostrador
df_mostrador = pd.read_csv(MOSTRADOR_SALIDA_PATH)

# Seleccionamos columnas y renombramos
df_resto_simple = df_resto[['fecha_emision', 'denominacion', 'TotalVenta_Ajustado','cuit_receptor','Anio','Mes','importe_total']].rename(columns={
    'fecha_emision': 'fecha',
    'denominacion': 'cliente',
    'TotalVenta_Ajustado': 'monto ajustado',
    'cuit_receptor': 'cuit',
    'Anio': 'anio',
    'Mes': 'mes',
    'importe_total': 'monto'
})

df_mostrador_simple = df_mostrador[['Fecha', 'Cliente', 'TotalVenta_Ajustado', 'Producto','CUIT','Anio','Mes','TotalVenta']].rename(columns={
    'Fecha': 'fecha',
    'Cliente': 'cliente',
    'TotalVenta_Ajustado': 'monto ajustado',
    'Producto': 'producto',
    'CUIT': 'cuit',
    'Anio': 'anio',
    'Mes': 'mes',
    'TotalVenta': 'monto'
})

# Asignamos origen 'servicio' a facturación
df_resto_simple['origen'] = 'servicio'

# Listas de productos (convertidas a minúsculas)
decoracion_items = ['ALMOHADON','APOYA CUCHARA','AZUCARERA','BANCO TRES PATAS', 'BANCO RECTANGULAR', 'COLGANTES DE CERAMICA','COLGANTES VARIOS','CORAZON CHICO','CORAZON GRANDE','CORAZON MEDIANO','CUENCO CHICO','CUENCO MEDIANO','CUENCO MINI','CUENCO GRANDE','JARRA','JARRITO/ MATE/ VASITO','JARRO','LECHERITA','MACETA CHICA','MACETA MEDIANA','PINGUINO LITRO','PINGUINO MEDIO LITRO','PLATO 24CM','PLATO 30CM','PLATO POSTRE','PORTA CUCHARA GRANDE','PORTA VELA','POSE TORTA','REGALERÍA','REGALERIA VARIOS','TAZA SOLA','TAZA SOLA ARTESANAL','TAZON ARTESANAL','TAZON PARA LANA','TELAS','TETERA','VELA']
decoracion_items = [item.lower() for item in decoracion_items]

restobar_items = ['RESTOBAR']
restobar_items = [item.lower() for item in restobar_items]

# Función para asignar origen
def clasificar_origen(producto):
    producto_lower = str(producto).lower()
    if any(item in producto_lower for item in decoracion_items):
        return 'decoracion'
    elif any(item in producto_lower for item in restobar_items):
        return 'restobar'
    else:
        return 'mostrador'

# Asignar origen a mostrador
df_mostrador_simple['origen'] = df_mostrador_simple['producto'].apply(clasificar_origen)

"""# Eliminamos la columna producto si no la querés en el final
df_mostrador_simple = df_mostrador_simple.drop(columns=['producto'])"""

# Concatenamos
df_unido = pd.concat([df_resto_simple, df_mostrador_simple], ignore_index=True)

# Ajustamos fecha
df_unido['fecha'] = df_unido['fecha'].astype(str).str.split().str[0]

# Exportamos
df_unido.to_csv(ARCHIVO_UNIFICADO_PATH, index=False)


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_400\2930985622.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mostrador = pd.read_csv(MOSTRADOR_SALIDA_PATH)
